# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [ ]:
!pip list

In [ ]:
!pip3 install keras\<2.3 -U

In [ ]:
!pip3 install numpy\<1.24 -U

In [ ]:
!mamba install -y pycocotools

# Init

In [1]:
#%tensorflow_version 1.x
#!export LD_PRELOAD=$LD_PRELOAD:/root/miniforge3/envs/mask/bin/../lib/libgomp.so.1
import os
#print(os.environ['LD_PRELOAD'])
#os.environ['LD_PRELOAD'] = "/root/miniforge3/envs/mask/lib/libgomp.so.1"
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

#%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print("ready..")

2024-04-10 19:09:53.918476: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


ready..


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
print("ready..")


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
print("ready..")

2024-04-10 19:10:07.023965: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 31250000 Hz
2024-04-10 19:10:07.026453: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xaaaae1bd8bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-10 19:10:07.026602: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-04-10 19:10:07.041055: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-04-10 19:10:07.177317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1049] ARM64 does not support NUMA - returning NUMA node zero
2024-04-10 19:10:07.177951: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xaaaae2612720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-10 19:10:07.178065: I tensorflow/compiler/xla/service/service.cc:176]   Stream

2024-04-10 19:10:08.287649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-04-10 19:10:08.287817: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 
2024-04-10 19:10:08.287866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N 
2024-04-10 19:10:08.288647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1049] ARM64 does not support NUMA - returning NUMA node zero
2024-04-10 19:10:08.289296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1049] ARM64 does not support NUMA - returning NUMA node zero
2024-04-10 19:10:08.289556: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 22432 MB memory) -> physical GPU (device: 0, name: Xavier, pci bus id: 0000:00:00.0, compute capability: 7.2)



ready..


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
print("ready..")

ready..


# Run Object Detection

In [ ]:
#from datetime 
import datetime
import time
#from mrcnn import visualize
total_fps=0
frame_count=0
MODEL_NAME_SUFFIX_IMG_NAMING="2017-11-26_04-33 matterport Mask_RCNN v2.0"
f = open(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt", "a")
for i in range(31):
# Load a random image from the images folder
    start_time_ms = time.time_ns() // 1_000_000
    file_names = next(os.walk(IMAGE_DIR))[2]
    image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
    start_forward_ms = time.time_ns() // 1_000_000
    results = model.detect([image], verbose=1)
    forward_end_time_ms = time.time_ns() // 1_000_000
    forward_pass_time_ms = forward_end_time_ms - start_forward_ms
    forward_pass_time = forward_pass_time_ms/1000
                # Get the current fps.
    fps = 1 / (forward_pass_time)
                # Add `fps` to `total_fps`.
    total_fps += fps
                # Increment frame count.
    frame_count += 1

# Visualize results
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], saveims=True)
    final_end_time_ms = time.time_ns() // 1_000_000
    forward_and_annot_time = (final_end_time_ms - start_time_ms)/1000
    print_string = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, Frame: {frame_count}, Forward pass FPS: {fps:.3f}, "
    print_string += f"Forward pass time: {forward_pass_time:.3f} seconds, "
    print_string += f"Forward pass + ann + disp time: {forward_and_annot_time:.3f} seconds\n"
    print(print_string)
    f.write(print_string)
avg_fps = total_fps / (frame_count if frame_count != 0 else 1)
print_string = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, Average FPS: {avg_fps:.3f}"
print(print_string)
f.write(print_string)
f.close()
!tmtmtm=$(date +"%Y-%m-%d_%H-%M-%S") && mkdir $tmtmtm && mv *.txt *.jpg $tmtmtm/

# Run Camera Object Detectoin

In [ ]:
#the cam
#%matplotlib notebook
%matplotlib inline
#%matplotlib ipympl
CAMERA_TIME_LIMIT_SECONDS = 28800
MODEL_NAME_SUFFIX_IMG_NAMING="2017-11-26_04-33 matterport Mask_RCNN v2.0"
import cv2
import asyncio
import time
import datetime
from mjpeg_streamer import MjpegServer, Stream
#720p = 1280 x 720
stream = Stream("r", size=(1280, 720), quality=50, fps=25)
server = MjpegServer("0.0.0.0", 1050)
server.add_stream(stream)
server.start()
cap = None
async def connect_camera():
    global cap
    print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, seeking the camera...")
    retr_count = 0
    while(True):
        try:
            cap = cv2.VideoCapture("http://jtag.me:2727")
            if not cap.isOpened():
                print_string = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, Error: Could not open MJPEG stream. waiting 1s to retry."
                if(retr_count <3):
                    print(print_string)
                    retr_count+= 1
                else:
                    log_print(print_string, print_string, "")
                    retr_count = 0
                time.sleep(1)
                continue
            print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, camera found!")
            break
        except Exception as ex:
            print(ex)
            print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, waiting 1s to retry.")
            time.sleep(1)
            ...

#thebreaker = True
fig, ax = plt.subplots(1, figsize=(14, 14))
plt.axis('off')
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)


CAMERA_BEGIN_TIME = time.time()
LAST_CAP_FRAME = None
async def cameraloop():
    global LAST_CAP_FRAME, cap, CAMERA_BEGIN_TIME, CAMERA_TIME_LIMIT_SECONDS, ax
    if (cap is None): await connect_camera()
    while(True):
        if(time.time()-CAMERA_BEGIN_TIME +3 >CAMERA_TIME_LIMIT_SECONDS): return
        #print("cameraloop")
        # GIL vēl aizvien ir te :(*)
        await asyncio.sleep(0)
        success, jpg = cap.read()
        if not success:
            print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, could not read frame.")
            time.sleep(1)
            await connect_camera()
            #connect_camera()
            continue
        else:
             LAST_CAP_FRAME = jpg

oneTime=True
async def detectloop():
    global LAST_CAP_FRAME, cap, CAMERA_BEGIN_TIME, CAMERA_TIME_LIMIT_SECONDS, ax, stream, oneTime
    #print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, detectentry")
    while(True):
        if(time.time()-CAMERA_BEGIN_TIME +3 >CAMERA_TIME_LIMIT_SECONDS): return
        if(LAST_CAP_FRAME is None):
            await asyncio.sleep(1)
            continue
        results = model.detect([LAST_CAP_FRAME])#, verbose=1)
        r = results[0]
             #ax.cla()
             #plt.clf()
        #print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, visualizing...")
        visualize.display_instances(LAST_CAP_FRAME, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], ax=ax, saveims=False)
        LAST_CAP_FRAME=None
             #fig1.canvas.flush_events()
        #w, h = fig.canvas.get_width_height()
        #img_np = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        #img_np = img_np.reshape((h, w, 3))
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} changing frame")
        if(oneTime):
            oneTime = False
            plt.show()
        fig = ax.get_figure()
        fig.canvas.draw()
        rgba = fig.canvas.renderer._renderer
        ax.cla()

        #stream.set_frame(img_np)
        stream.set_frame(np.array(rgba))
            
        #plt.show()

async def the_loop():
    print(f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} Mdl: {MODEL_NAME_SUFFIX_IMG_NAMING}, launching loops...")
    tasks = [
        cameraloop(),
        detectloop()
    ]
    await asyncio.gather(*tasks)

#await the_loop()
#asyncio.run(the_loop())
#connect_camera()
#cameraloop()

tasks = [
        cameraloop(),
        detectloop()
    ]
await asyncio.gather(*tasks)
server.stop()
#cap.release()
#cv2.destroyAllWindows()


Streams index: http://127.0.0.1:1050
Available streams:

http://127.0.0.1:1050/r
--------------------------------


Streams index: http://10.17.33.244:1050
Available streams:

http://10.17.33.244:1050/r
--------------------------------


Streams index: http://172.17.0.1:1050
Available streams:

http://172.17.0.1:1050/r
--------------------------------


Press Ctrl+C to stop the server

2024-04-10_19-10-39 Mdl: 2017-11-26_04-33 matterport Mask_RCNN v2.0, seeking the camera...
2024-04-10_19-10-40 Mdl: 2017-11-26_04-33 matterport Mask_RCNN v2.0, camera found!


2024-04-10 19:10:45.421452: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-04-10 19:10:46.485724: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


In [ ]:
%tb